In [4]:
path = "/content/drive/MyDrive/Nittany AI/Colab Notebooks/RAG/messi.txt"

# Read the file as a list of lines
with open(path, "r", encoding="utf-8") as file:
    messi_data = file.readlines()

# Convert to a DataFrame for structured processing
messi_df = pd.DataFrame(messi_data, columns=["text"])

# Display first few rows
print(messi_df.head())

                                                text
0  Lionel Andrés Messi[note 1] (Spanish pronuncia...
1                                                 \n
2  Born in Rosario, Argentina, Messi relocated to...
3                                                 \n
4  An Argentine international, Messi is the natio...


In [5]:
messi_texts = messi_df["text"].tolist()

from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')
messi_embeddings = model.encode(messi_texts, convert_to_tensor=True)

from sentence_transformers import util

# User's question
question = "How many Ballon d'Or awards has Messi won?"
question_embedding = model.encode(question, convert_to_tensor=True)

# Compute cosine similarity
similarities = util.cos_sim(question_embedding, messi_embeddings)[0]

# Get top matches
top_k = 3
top_k_indices = similarities.topk(k=top_k).indices

# Retrieve relevant lines
relevant_context = [messi_texts[i] for i in top_k_indices]
print("Relevant Context:", relevant_context)

Relevant Context: ["Born in Rosario, Argentina, Messi relocated to Spain to join Barcelona at age 13, and made his competitive debut at age 17 in October 2004. He established himself as an integral player for the club within the next three years, and in his first uninterrupted season in 2008–09 helped Barcelona achieve the first treble in Spanish football; that year, aged 22, Messi won the first of his four consecutive Ballons d'Or, the first player to win it four times. During the 2011–12 season, he set La Liga and European records for most goals in a season, while establishing himself as Barcelona's all-time top scorer. The following two seasons, he finished second for the Ballon d'Or behind Cristiano Ronaldo, his perceived career rival, before regaining his best form during the 2014–15 campaign, where he became the all-time top scorer in La Liga, led Barcelona to a historic second treble, and won a fifth Ballon d'Or in 2015. Messi assumed captaincy of Barcelona in 2018, and won a re

In [6]:
import openai

# Ensure you have an OpenAI API Key set
openai.api_key = "your_openai_api_key"

def query_llm_with_context(question, context):
    """
    Query the LLM (e.g., OpenAI's GPT-4) using the retrieved context.
    """
    prompt = f"""
    Using the following context, answer the question concisely and accurately:

    Context:
    {''.join(context)}

    Question:
    {question}
    """

    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are an expert on Lionel Messi."},
            {"role": "user", "content": prompt}
        ]
    )
    return response['choices'][0]['message']['content']

# Example usage:
llm_answer = query_llm_with_context(question, relevant_context)
print("Answer:", llm_answer)

APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742
